### Settings & imports

In [1]:
%matplotlib notebook

In [2]:
import sys
mypath = "/home/basia/Documents/spectroscopy/wassersteinms"
sys.path.insert(0, mypath)
import masserstein 

In [3]:
import os

from masserstein import Spectrum, NMRSpectrum
from masserstein import estimate_proportions

from matplotlib import pyplot as plt

import numpy as np

import pulp

import pickle

import seaborn as sns
from textwrap import wrap

from pathlib import Path

import pandas as pd

In [4]:
nr_of_experiment = 9

### Experiments data

In [5]:
components_dictionary = {'experiment_1': ['Pinene', 'Benzyl_benzoate'],
                        'experiment_2': ['Pinene', 'Limonene'],
                        'experiment_3': ['Benzyl_benzoate', 'm Anisaldehyde'],
                        'experiment_4': ['Isopropyl_myristate', 'Benzyl_benzoate', 'Alpha_pinene', 'Limonene'],
                        'experiment_5': ['Pinene', 'Benzyl_benzoate'],
                        'experiment_6': ['Lactate', 'Alanine', 'Creatine', 'Creatinine', 'Choline chloride'],
                        'experiment_7': ['Lactate', 'Alanine', 'Creatine', 'Creatinine', 'Choline chloride'],
                        'experiment_8': ['Lactate', 'Alanine', 'Creatine', 'Creatinine', 'Choline chloride'],
                        'experiment_9': ['Benzyl_benzoate', 'm Anisaldehyde']}

In [6]:
protons_dictionary = {'experiment_1': [16, 12],
                        'experiment_2': [16, 16],
                        'experiment_3': [12, 8],
                        'experiment_4': [34, 12, 16, 16],
                        'experiment_5': [16, 12],
                        'experiment_6': [4, 4, 5, 5, 13],
                        'experiment_7': [4, 4, 5, 5, 13],
                        'experiment_8': [4, 4, 5, 5, 13],
                        'experiment_9': [12, 8]}

In [7]:
ground_truth_molar_proportions = {'experiment_1': [0.09088457406472417, 0.9091154259352758],
                                    'experiment_2': [0.505, 0.495],
                                    'experiment_3': [0.8403875207510383, 0.1596124792489616],
                                    'experiment_4': [0.7264578344443725, 0.10578603326645526, 0.081968804608116, 0.08578732768105625],
                                    'experiment_5': [0.3865, 0.6135],
                                    'experiment_6': [0.3022, 0.2240, 0.1253, 0.2028, 0.1457],
                                    'experiment_7': [0.3022, 0.2240, 0.1253, 0.2028, 0.1457],
                                    'experiment_8': [0.3022, 0.2240, 0.1253, 0.2028, 0.1457],
                                    'experiment_9': [0.3702, 0.6298]}

In [8]:
experiments_folders = {'experiment_1': 'experiment_1_intensity_difference',
                        'experiment_5': 'experiment_5_miniperfumes',
                        'experiment_6': 'experiment_6_metabolites',
                         'experiment_3': 'experiment_3_overlapping_and_intensity_difference',
                         'experiment_7': 'experiment_7_and_8_shim',
                       'experiment_8': 'experiment_7_and_8_shim',
                        'experiment_9': 'experiment_9_different_solvents',
                        'experiment_4': 'experiment_4_perfumes_and_absent_components',
                         'experiment_2': 'experiment_2_overlapping'}

In [9]:
best_kappa_mix = [0.09, 0.05, 0.23, 0.37, 0.06, 0.03, 0.05, 0.11, 0.2]

In [10]:
best_kappa_model = [0.08, 0.01, 0.02, 0.24, 0.2, 0.09, 0.02, 0.02, 0.29]

### Loading the data

In [11]:
cwd = Path(os.getcwd())
path = cwd.parent.absolute()

In [12]:
if (nr_of_experiment != 7 and nr_of_experiment != 8):
    filename = str(path)+'/' + experiments_folders['experiment_' + str(nr_of_experiment)] + '/preprocessed_mix.csv'
    mix = np.loadtxt(filename, delimiter=',')
else:
    filename = str(path) + '/' + experiments_folders['experiment_' + str(nr_of_experiment)] + '/preprocessed_exp'+str(nr_of_experiment)+'_mix.csv'
    mix = np.loadtxt(filename, delimiter=',')

In [13]:
how_many_components = len(components_dictionary['experiment_'+str(nr_of_experiment)])
names = ['comp' + str(i) for i in range(how_many_components)]

In [14]:
spectra = []
for i in range(how_many_components):
    if (nr_of_experiment != 7 and nr_of_experiment != 8):
        filename = str(path) + '/' + experiments_folders['experiment_' + str(nr_of_experiment)] + \
                    '/preprocessed_comp' + str(i) + '.csv'
        spectra.append(np.loadtxt(filename, delimiter=','))
    else:
        filename = str(path) + '/' + experiments_folders['experiment_' + str(nr_of_experiment)] + \
                    '/preprocessed_exp'+str(nr_of_experiment)+ '_comp'+ str(i) + '.csv'
        spectra.append(np.loadtxt(filename, delimiter=','))

In [15]:
spectra2 = []
names = []
for i in range(len(spectra)):
    spectra2.append(NMRSpectrum(confs=list(zip(spectra[i][:,0], spectra[i][:,1])), 
                                protons=protons_dictionary['experiment_'+str(nr_of_experiment)][i]))
    names.append('comp'+str(i))

spectra = spectra2
del(spectra2)
mix = NMRSpectrum(confs=list(zip(mix[:,0], mix[:,1])))

In [16]:
for i, name in enumerate(names):
    globals()[name] = spectra[i]

In [17]:
# mix.plot(profile=True)
# comp0.plot(profile=True)
# comp1.plot(profile=True)

### Estimation

In [18]:
estimation_vis = estimate_proportions(mix, spectra,
                                        MTD=best_kappa_mix[nr_of_experiment-1], 
                                        MTD_th=best_kappa_model[nr_of_experiment-1],
                                        verbose=True, solver=pulp.GUROBI(msg=False), 
                                        what_to_compare='area')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10


Initial filtering of formulas: 100%|██████████████| 2/2 [00:00<00:00, 54.10it/s]


Removed theoretical spectra due to no matching experimental peaks: []
Envelope bounds: [(3.5000299999999998, 11.99995, 0), (3.5000299999999998, 11.99995, 1)]


Computing chunks: 100%|████████████████████████| 2/2 [00:00<00:00, 66052.03it/s]


Number of chunks: 1
ChunkIDs: [0, 0]
Chunk bounds: [(3.2100299999999997, 12.28995)]


Splitting the experimental spectrum into chunks: 139577it [00:00, 3593017.89it/s]


Ion currents in chunks: [0.9999999999999973]


Deconvolving chunks:   0%|                                | 0/1 [00:00<?, ?it/s]

Deconvolving chunk 0


/home/basia/.local/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
Deconvolving chunks: 100%|███████████████████████| 1/1 [03:12<00:00, 192.21s/it]

Chunk %i deconvolution status: Optimal
Signal proportion in experimental spectrum: 0.9298597180920001
Noise proportion in experimental spectrum: 0.07014029297699982
Total explanation: 1.000000011069
Noise proportion in combination of theoretical spectra: 0.008558474193


In [19]:
estimation_vis['proportions']

[0.43970247519399885, 0.49015724289799867]

In [21]:
estimation_vis['proportion_of_noise_in_theoretical']

0.008558474192999977

In [21]:
estimation_molar = estimate_proportions(mix, spectra,
                                        MTD=best_kappa_mix[nr_of_experiment-1], 
                                        MTD_th=best_kappa_model[nr_of_experiment-1],
                                        verbose=True, solver=pulp.GUROBI(msg=False), 
                                        what_to_compare='concentration')

Initial filtering of formulas: 100%|█████████████| 5/5 [00:00<00:00, 299.96it/s]


Removed theoretical spectra due to no matching experimental peaks: []
Envelope bounds: [(0.827366, 4.1272, 0), (0.827366, 4.1272, 1), (0.827366, 4.1272, 2), (0.827366, 4.1272, 3), (0.827366, 4.1272, 4)]


Computing chunks: 100%|███████████████████████| 5/5 [00:00<00:00, 209715.20it/s]


Number of chunks: 1
ChunkIDs: [0, 0, 0, 0, 0]
Chunk bounds: [(0.7173660000000001, 4.2372000000000005)]


Splitting the experimental spectrum into chunks: 27129it [00:00, 3144424.05it/s]


Ion currents in chunks: [0.9999999999999996]


Deconvolving chunks:   0%|                                | 0/1 [00:00<?, ?it/s]

Deconvolving chunk 0


Deconvolving chunks: 100%|████████████████████████| 1/1 [00:12<00:00, 12.87s/it]

Chunk %i deconvolution status: Optimal
Signal proportion in experimental spectrum: 1.0
Noise proportion in experimental spectrum: 0.0
Total explanation: 1.0
Noise proportion in combination of theoretical spectra: 0.024876849655


In [22]:
estimation_molar['proportions']

[0.31770096865088626,
 0.21196331538085228,
 0.13209747879635073,
 0.197528802988393,
 0.14070943418351764]

### Transport plan histograms

In [22]:
components_ints = [np.array(sp.confs)[:,1] for sp in spectra]
components_ppm = np.array(spectra[0].confs)[:,0]

In [23]:
ppm = np.array(mix.confs)[:,0]
mix_ints = np.array(mix.confs)[:,1]

In [24]:
assert np.all(ppm == components_ppm)
assert np.all(ppm == estimation_vis['global_mass_axis'])

#### Removing signal that went to vortex

##### Mixture

In [25]:
mix_without_noise = NMRSpectrum(confs = list(zip(ppm, mix_ints - np.array(estimation_vis['noise']))))

##### Components

In [26]:
p = np.array(estimation_vis['proportions']).reshape(len(estimation_vis['proportions']),1)

In [27]:
names_matmul = ['sum_up_to_'+str(i+1) for i in range(len(spectra))]

In [28]:
components_ints = [comp_int.reshape(-1, 1) for comp_int in components_ints]

In [29]:
X = np.concatenate(components_ints, axis=1)

In [30]:
for i, name in enumerate(names_matmul):
    globals()[name] = np.dot(X[:,:(i+1)],p[:(i+1),:])[:,0]

In [31]:
mix_estimation = np.concatenate([ppm.reshape(-1,1), globals()[names_matmul[-1]].reshape(-1,1)], axis=1)

In [32]:
components_without_noise = NMRSpectrum(confs = list(zip(mix_estimation[:,0], mix_estimation[:,1] - np.array(estimation_vis['noise_in_theoretical']))))

In [33]:
# components_without_noise.plot(profile=True)
# mix_without_noise.plot(profile=True)

In [34]:
components_without_noise.trim_negative_intensities()
mix_without_noise.trim_negative_intensities()

In [35]:
components_without_noise.normalize()
mix_without_noise.normalize()

In [36]:
# components_without_noise.plot(profile=True)
# mix_without_noise.plot(profile=True)

#### Histograms

In [37]:
mix_without_noise.WSDistance(components_without_noise)

0.2703154449229968

In [38]:
distances = [abs(x[0]-x[1]) for x in list(mix_without_noise.WSDistanceMoves(components_without_noise))]
signals = [x[2] for x in list(mix_without_noise.WSDistanceMoves(components_without_noise))]

In [39]:
distances_signals = list(zip(distances, signals))
distances_signals = pd.DataFrame(distances_signals, columns = ['Distances', 'Signals'])
distances_signals = distances_signals.groupby('Distances').agg('sum')
distances_signals.sort_index(inplace=True)

In [40]:
MTD=best_kappa_mix[nr_of_experiment-1]
MTD_th=best_kappa_model[nr_of_experiment-1]

<IPython.core.display.Javascript object>


In [41]:
H, bins = distances_signals['Signals'], distances_signals.index

text_position = [0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005]
text_position_th = [-0.015, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005]
xlims = [0.4, 0.1, 0.3, 0.5, 0.3, 0.3, 0.15, 0.15, 0.4]
xlims = [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]

fig, ax = plt.subplots()
#ax.set_yscale("log")
ax.set_xlim(-0.001, xlims[nr_of_experiment-1])
ax.set_ylim(0, 0.05)

ax.bar(bins, H, width = 0.005, color='#707070')

plt.xlabel('Transport distance [ppm]', fontsize=15, labelpad=15)
plt.ylabel('Amount of signal transported', fontsize=15, labelpad=15)

plt.axvline(x=MTD, color='#7EBCE1')
plt.text(MTD + text_position[nr_of_experiment-1], 0.01,'Kappa mixture',rotation=90, color='#7EBCE1')

plt.axvline(x=MTD_th, color='#A35FC1', linestyle='--')
plt.text(MTD_th+ text_position_th[nr_of_experiment-1], 0.01,'Kappa components',rotation=90, color='#A35FC1')

plt.tight_layout()
plt.show()
plt.savefig('distances_histogram_exp_'+str(nr_of_experiment)+'.png', dpi=300)

In [44]:
percent = 0.99
q = H.quantile(q=percent)
print(str(percent*100)+'% of signal is transported on distance smaller than '+str(q)+'.' )

99.0% of signal is transported on distance smaller than 0.00012810135156687014.
